In [1]:
import os
from dotenv import load_dotenv

from src.datasetgenerator import StratzQuery, DatasetGenerator




In [2]:
load_dotenv()
#querier = StratzQuery(os.getenv('STRATZ_TOKEN'))
#querier.get_match(7590822094)

csv_path = os.getenv('CSV_PATH')

generator = DatasetGenerator(os.getenv('STRATZ_TOKEN'), csv_path)

In [6]:

league = generator.add_professional_league(16842)
matches = league['matches']
len(matches)

43

In [8]:
matches[0].keys()

dict_keys(['didRadiantWin', 'durationSeconds', 'startDateTime', 'firstBloodTime', 'averageRank', 'players', 'playbackData', 'averageImp', 'actualRank', 'radiantKills', 'direKills'])

In [15]:
matches[0]['playbackData']['wardEvents']

[{'indexId': 450,
  'time': -63,
  'positionX': 126,
  'positionY': 126,
  'fromPlayer': 4,
  'wardType': 'OBSERVER',
  'action': 'SPAWN',
  'playerDestroyed': None},
 {'indexId': 454,
  'time': -62,
  'positionX': 154,
  'positionY': 100,
  'fromPlayer': 2,
  'wardType': 'OBSERVER',
  'action': 'SPAWN',
  'playerDestroyed': None},
 {'indexId': 459,
  'time': -53,
  'positionX': 120,
  'positionY': 122,
  'fromPlayer': 128,
  'wardType': 'OBSERVER',
  'action': 'SPAWN',
  'playerDestroyed': None},
 {'indexId': 517,
  'time': -17,
  'positionX': 138,
  'positionY': 82,
  'fromPlayer': 129,
  'wardType': 'OBSERVER',
  'action': 'SPAWN',
  'playerDestroyed': None},
 {'indexId': 548,
  'time': -9,
  'positionX': 128,
  'positionY': 74,
  'fromPlayer': 130,
  'wardType': 'SENTRY',
  'action': 'SPAWN',
  'playerDestroyed': None},
 {'indexId': 854,
  'time': 14,
  'positionX': 168,
  'positionY': 92,
  'fromPlayer': 2,
  'wardType': 'SENTRY',
  'action': 'SPAWN',
  'playerDestroyed': None},
 

In [89]:
from collections import defaultdict

def get_match_wards(match:dict) -> dict:
    ward_events = match['playbackData']['wardEvents']
    wards = defaultdict(lambda: {})
    for w_e in ward_events:
        if w_e["action"] == "SPAWN":
            wards[w_e['indexId']] = {
                "indexId": w_e["indexId"],
                "spawned_time": w_e["time"],
                "despawned_time": None,
                "positionX": w_e["positionX"],
                "positionY": w_e["positionY"],
                "wardType": w_e["wardType"],
                "fromPlayer": w_e["fromPlayer"],
                "playerDestroyed": w_e["playerDestroyed"],
            }
            continue
        wards[w_e['indexId']]["despawned_time"] = w_e['time']
    return dict(wards)

wards = get_match_wards(matches[0])
wards.keys(), len(wards.keys())

(dict_keys([450, 454, 459, 517, 548, 854, 1877, 1884, 2217, 2224, 2491, 2257, 881, 2227, 2383, 2400, 1656, 2215, 1950, 1683, 1126, 2487, 2146, 2457, 2405, 2047, 2425, 2451, 2088, 1040, 2551, 814, 2226, 2177, 2128, 1918, 2480, 2209, 856, 1848, 920, 1598, 747, 2508, 1720, 2529, 857, 1862, 1554, 2090, 2422, 1672, 541, 921, 1054, 2361, 2397, 1743, 2554, 866, 228, 261, 230, 2132, 2447, 2170, 746, 2152, 848, 2049, 1773, 2038, 2338, 1925, 1858, 1599, 1566, 1810, 2396, 2331, 2079, 2534, 1959, 231, 1926, 2134, 2308, 1768, 2273, 2406, 879, 1631, 452, 461, 775, 1624, 1668, 1713, 1736, 1876, 1775]),
 101)

In [92]:
from collections import defaultdict
import pandas as pd

def get_df_match_wards(match:dict) -> dict:
    wards = get_match_wards(match)
    df_wards = defaultdict(lambda: [])
    for id, w in wards.items():
        for key, item in w.items():
            df_wards[key].append(item)
    df_wards = pd.DataFrame.from_dict(df_wards).set_index("indexId")
    df_wards['spawned_time'] = df_wards['spawned_time'] // 60
    df_wards['despawned_time'] = df_wards['despawned_time'] // 60
    return df_wards

df_wards = get_df_match_wards(matches[0])
df_wards

,spawned_time,despawned_time,positionX,positionY,wardType,fromPlayer,playerDestroyed
indexId,,,,,,,
450,-2,4,126,126,OBSERVER,4,None
454,-2,4,154,100,OBSERVER,2,None
459,-1,5,120,122,OBSERVER,128,None
517,-1,5,138,82,OBSERVER,129,None
548,-1,0,128,74,SENTRY,130,None
...,...,...,...,...,...,...,...
1668,38,40,174,148,SENTRY,2,None
1713,39,39,176,152,SENTRY,130,None
1736,39,40,118,138,OBSERVER,130,None


In [38]:
wards[450]

{'spawned_time': -63,
 'despawned_time': 296,
 'positionX': 126,
 'positionY': 126,
 'wardType': 'OBSERVER',
 'fromPlayer': 4,
 'playerDestroyed': None}

In [83]:
def get_match_wards_by_minute():
    wards = get_match_wards(matches[0])
    times = [w['despawned_time'] if w['despawned_time'] else w['spawned_time'] for i, w in wards.items()]
    max_time = max(times)
    wards_by_time = {
        str(t): []
        for t in range(-2, max_time // 60, 1)
    }
    for id, w in wards.items():
        w_spawned_t = w['spawned_time']
        w_despawned_t = w['despawned_time'] if w['despawned_time'] else max_time
        for t in range(w_spawned_t // 60, w_despawned_t  // 60, 1):
            wards_by_time[str(t)].append(w)
    return wards_by_time

ward_by_min = get_match_wards_by_minute()
ward_by_min

{'-2': [{'spawned_time': -63,
   'despawned_time': 296,
   'positionX': 126,
   'positionY': 126,
   'wardType': 'OBSERVER',
   'fromPlayer': 4,
   'playerDestroyed': None},
  {'spawned_time': -62,
   'despawned_time': 297,
   'positionX': 154,
   'positionY': 100,
   'wardType': 'OBSERVER',
   'fromPlayer': 2,
   'playerDestroyed': None}],
 '-1': [{'spawned_time': -63,
   'despawned_time': 296,
   'positionX': 126,
   'positionY': 126,
   'wardType': 'OBSERVER',
   'fromPlayer': 4,
   'playerDestroyed': None},
  {'spawned_time': -62,
   'despawned_time': 297,
   'positionX': 154,
   'positionY': 100,
   'wardType': 'OBSERVER',
   'fromPlayer': 2,
   'playerDestroyed': None},
  {'spawned_time': -53,
   'despawned_time': 306,
   'positionX': 120,
   'positionY': 122,
   'wardType': 'OBSERVER',
   'fromPlayer': 128,
   'playerDestroyed': None},
  {'spawned_time': -17,
   'despawned_time': 342,
   'positionX': 138,
   'positionY': 82,
   'wardType': 'OBSERVER',
   'fromPlayer': 129,
   'p

In [86]:
for t, wards_t in ward_by_min.items():


-2 [{'spawned_time': -63, 'despawned_time': 296, 'positionX': 126, 'positionY': 126, 'wardType': 'OBSERVER', 'fromPlayer': 4, 'playerDestroyed': None}, {'spawned_time': -62, 'despawned_time': 297, 'positionX': 154, 'positionY': 100, 'wardType': 'OBSERVER', 'fromPlayer': 2, 'playerDestroyed': None}]
-1 [{'spawned_time': -63, 'despawned_time': 296, 'positionX': 126, 'positionY': 126, 'wardType': 'OBSERVER', 'fromPlayer': 4, 'playerDestroyed': None}, {'spawned_time': -62, 'despawned_time': 297, 'positionX': 154, 'positionY': 100, 'wardType': 'OBSERVER', 'fromPlayer': 2, 'playerDestroyed': None}, {'spawned_time': -53, 'despawned_time': 306, 'positionX': 120, 'positionY': 122, 'wardType': 'OBSERVER', 'fromPlayer': 128, 'playerDestroyed': None}, {'spawned_time': -17, 'despawned_time': 342, 'positionX': 138, 'positionY': 82, 'wardType': 'OBSERVER', 'fromPlayer': 129, 'playerDestroyed': None}, {'spawned_time': -9, 'despawned_time': 15, 'positionX': 128, 'positionY': 74, 'wardType': 'SENTRY', '